In [14]:
import pandas as pd
import geopandas as gpd
import movingpandas as mpd
import pickle
import hvplot.pandas
import matplotlib.pyplot as plt

In [15]:
pandas = pd.read_csv('../sample/buffer_2-animals.csv',parse_dates=['timestamp'])
# see `../sample/meta.csv"
projection = "+proj=longlat +datum=WGS84 +no_defs"
timezone = "Africa/Dar_es_Salaam"
pandas['timestamp_tz'] = pandas['timestamp'].apply(lambda x: x.tz_localize(timezone))

timestamps = pandas[["timestamp", "timestamp_tz"]]
# timestamps.head()
pandas

,event.id,timestamp,location.long,location.lat,heading,height.above.ellipsoid,migration.stage,migration.stage.standard,coords.x1,coords.x2,...,timestamp.start,timestamp.end,number.of.events,number.of.deployments,sensor.type.ids,taxon.detail,animalName,heading.1,height.above.ellipsoid.1,timestamp_tz
0,1149963393,2013-08-28 18:00:41,48.775804,69.135516,0.00,108.4,breedmoult,NaN,48.775804,69.135516,...,NaN,NaN,0,0,NaN,NaN,3089_139537871,NaN,NaN,2013-08-28 18:00:41+03:00
1,1149963394,2013-08-29 06:00:23,49.237201,68.959293,0.00,35.9,flight,NaN,49.237201,68.959293,...,NaN,NaN,0,0,NaN,NaN,3089_139537871,NaN,NaN,2013-08-29 06:00:23+03:00
2,1149963395,2013-08-29 06:20:08,49.237163,68.959343,332.92,40.2,flight,NaN,49.237163,68.959343,...,NaN,NaN,0,0,NaN,NaN,3089_139537871,NaN,NaN,2013-08-29 06:20:08+03:00
3,1149963396,2013-08-29 06:40:07,49.236849,68.959235,152.70,44.8,flight,NaN,49.236849,68.959235,...,NaN,NaN,0,0,NaN,NaN,3089_139537871,NaN,NaN,2013-08-29 06:40:07+03:00
4,1149963397,2013-08-29 07:00:07,49.236658,68.959162,0.00,36.9,flight,NaN,49.236658,68.959162,...,NaN,NaN,0,0,NaN,NaN,3089_139537871,NaN,NaN,2013-08-29 07:00:07+03:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
843,1150000033,2014-04-11 02:30:26,23.293136,55.726604,NaN,NaN,stopover,spring-migration,23.293136,55.726604,...,NaN,NaN,0,0,NaN,NaN,3079_139537881,NaN,NaN,2014-04-11 02:30:26+03:00
844,1150000034,2014-04-11 06:30:25,23.174233,55.735145,NaN,NaN,stopover,spring-migration,23.174233,55.735145,...,NaN,NaN,0,0,NaN,NaN,3079_139537881,NaN,NaN,2014-04-11 06:30:25+03:00
845,1150000035,2014-04-11 10:30:54,23.352401,55.858963,NaN,NaN,flight,spring-migration,23.352401,55.858963,...,NaN,NaN,0,0,NaN,NaN,3079_139537881,NaN,NaN,2014-04-11 10:30:54+03:00
846,1150000036,2014-04-11 11:00:27,23.356507,55.860330,NaN,NaN,flight,spring-migration,23.356507,55.860330,...,NaN,NaN,0,0,NaN,NaN,3079_139537881,NaN,NaN,2014-04-11 11:00:27+03:00


In [16]:
movingpandas = mpd.TrajectoryCollection(
            pandas,
            traj_id_col='individual.id',
            crs=projection,
            t='timestamp_tz',  # use our converted timezone column
            x='location.long',
            y='location.lat'
        )
movingpandas.hvplot()

/Users/clemens/mambaforge/envs/link-r-python/lib/python3.9/site-packages/movingpandas/trajectory_collection.py:155: FutureWarning: In a future version of pandas, a length 1 tuple will be returned when iterating over a groupby with a grouper equal to a list of length 1. Don't supply a list with a single grouper to avoid this warning.
  for traj_id, values in df.groupby([traj_id_col]):
/Users/clemens/mambaforge/envs/link-r-python/lib/python3.9/site-packages/geopandas/array.py:1406: UserWarning: CRS not set for some of the concatenation inputs. Setting output's CRS as unknown (the single non-null crs provided).
  warnings.warn(
/Users/clemens/mambaforge/envs/link-r-python/lib/python3.9/site-packages/shapely/constructive.py:181: RuntimeWarning: invalid value encountered in buffer
  return lib.buffer(
/Users/clemens/mambaforge/envs/link-r-python/lib/python3.9/site-packages/shapely/set_operations.py:133: RuntimeWarning: invalid value encountered in intersection
  return lib.intersection(a, b

:Overlay
   .Tiles.I :Tiles   [x,y]
   .Path.I  :Path   [Longitude,Latitude]
   .Path.II :Path   [Longitude,Latitude]

In [17]:
geopandas = movingpandas.to_traj_gdf()
geopandas

,traj_id,start_t,end_t,geometry,length,direction
0,138936205,2013-09-10 04:41:02,2014-04-11 11:30:25,"LINESTRING (48.96278 67.58143, 48.96626 67.586...",4.462412e+06,237.922821
1,138936208,2013-08-28 18:00:41,2013-09-29 18:20:06,"LINESTRING (48.77580 69.13552, 49.23720 68.959...",3.243802e+06,244.167229


In [18]:
geopandas_lines = movingpandas.to_line_gdf()
geopandas_lines

/Users/clemens/mambaforge/envs/link-r-python/lib/python3.9/site-packages/geopandas/array.py:1406: UserWarning: CRS not set for some of the concatenation inputs. Setting output's CRS as unknown (the single non-null crs provided).
  warnings.warn(
/Users/clemens/mambaforge/envs/link-r-python/lib/python3.9/site-packages/geopandas/array.py:1406: UserWarning: CRS not set for some of the concatenation inputs. Setting output's CRS as unknown (the single non-null crs provided).
  warnings.warn(


,event.id,timestamp,heading,height.above.ellipsoid,migration.stage,migration.stage.standard,coords.x1,coords.x2,optional,sensor,...,number.of.events,number.of.deployments,sensor.type.ids,taxon.detail,animalName,heading.1,height.above.ellipsoid.1,t,prev_t,geometry
0,1149959537,2013-09-10 06:00:45,NaN,NaN,stopover,fall-migration,48.966255,67.586608,True,GPS,...,0,0,NaN,NaN,3079_139537881,NaN,NaN,2013-09-10 06:00:45,2013-09-10 04:41:02,"LINESTRING (48.96278 67.58143, 48.96626 67.58661)"
1,1149959538,2013-09-10 07:40:45,NaN,NaN,stopover,fall-migration,48.968824,67.583088,True,GPS,...,0,0,NaN,NaN,3079_139537881,NaN,NaN,2013-09-10 07:40:45,2013-09-10 06:00:45,"LINESTRING (48.96626 67.58661, 48.96882 67.58309)"
2,1149959539,2013-09-10 09:00:45,NaN,NaN,stopover,fall-migration,48.966683,67.582436,True,GPS,...,0,0,NaN,NaN,3079_139537881,NaN,NaN,2013-09-10 09:00:45,2013-09-10 07:40:45,"LINESTRING (48.96882 67.58309, 48.96668 67.58244)"
3,1149959540,2013-09-10 10:30:49,NaN,NaN,stopover,fall-migration,48.965634,67.582531,True,GPS,...,0,0,NaN,NaN,3079_139537881,NaN,NaN,2013-09-10 10:30:49,2013-09-10 09:00:45,"LINESTRING (48.96668 67.58244, 48.96563 67.58253)"
4,1149959541,2013-09-10 12:00:45,NaN,NaN,stopover,fall-migration,48.962427,67.581130,True,GPS,...,0,0,NaN,NaN,3079_139537881,NaN,NaN,2013-09-10 12:00:45,2013-09-10 10:30:49,"LINESTRING (48.96563 67.58253, 48.96243 67.58113)"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
841,1149963611,2013-09-29 17:00:07,0.00,73.7,flight,NaN,12.588869,52.667989,True,GPS,...,0,0,NaN,NaN,3089_139537871,NaN,NaN,2013-09-29 17:00:07,2013-09-29 16:40:06,"LINESTRING (12.58889 52.66796, 12.58887 52.66799)"
842,1149963612,2013-09-29 17:20:06,237.24,72.5,flight,NaN,12.588875,52.667995,True,GPS,...,0,0,NaN,NaN,3089_139537871,NaN,NaN,2013-09-29 17:20:06,2013-09-29 17:00:07,"LINESTRING (12.58887 52.66799, 12.58887 52.66799)"
843,1149963613,2013-09-29 17:40:06,259.52,228.7,flight,NaN,12.483441,52.607326,True,GPS,...,0,0,NaN,NaN,3089_139537871,NaN,NaN,2013-09-29 17:40:06,2013-09-29 17:20:06,"LINESTRING (12.58887 52.66799, 12.48344 52.60733)"
844,1149963614,2013-09-29 18:00:06,133.04,148.9,flight,NaN,12.313063,52.558519,True,GPS,...,0,0,NaN,NaN,3089_139537871,NaN,NaN,2013-09-29 18:00:06,2013-09-29 17:40:06,"LINESTRING (12.48344 52.60733, 12.31306 52.55852)"


In [19]:
geopandas_point = movingpandas.to_point_gdf()
geopandas_point

,event.id,timestamp,heading,height.above.ellipsoid,migration.stage,migration.stage.standard,coords.x1,coords.x2,optional,sensor,...,timestamp.start,timestamp.end,number.of.events,number.of.deployments,sensor.type.ids,taxon.detail,animalName,heading.1,height.above.ellipsoid.1,geometry
timestamp_tz,,,,,,,,,,,,,,,,,,,,,
2013-09-10 04:41:02,1149959536,2013-09-10 04:41:02,NaN,NaN,stopover,fall-migration,48.962778,67.581425,True,GPS,...,NaN,NaN,0,0,NaN,NaN,3079_139537881,NaN,NaN,POINT (48.96278 67.58143)
2013-09-10 06:00:45,1149959537,2013-09-10 06:00:45,NaN,NaN,stopover,fall-migration,48.966255,67.586608,True,GPS,...,NaN,NaN,0,0,NaN,NaN,3079_139537881,NaN,NaN,POINT (48.96626 67.58661)
2013-09-10 07:40:45,1149959538,2013-09-10 07:40:45,NaN,NaN,stopover,fall-migration,48.968824,67.583088,True,GPS,...,NaN,NaN,0,0,NaN,NaN,3079_139537881,NaN,NaN,POINT (48.96882 67.58309)
2013-09-10 09:00:45,1149959539,2013-09-10 09:00:45,NaN,NaN,stopover,fall-migration,48.966683,67.582436,True,GPS,...,NaN,NaN,0,0,NaN,NaN,3079_139537881,NaN,NaN,POINT (48.96668 67.58244)
2013-09-10 10:30:49,1149959540,2013-09-10 10:30:49,NaN,NaN,stopover,fall-migration,48.965634,67.582531,True,GPS,...,NaN,NaN,0,0,NaN,NaN,3079_139537881,NaN,NaN,POINT (48.96563 67.58253)
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2013-09-29 17:00:07,1149963611,2013-09-29 17:00:07,0.00,73.7,flight,NaN,12.588869,52.667989,True,GPS,...,NaN,NaN,0,0,NaN,NaN,3089_139537871,NaN,NaN,POINT (12.58887 52.66799)
2013-09-29 17:20:06,1149963612,2013-09-29 17:20:06,237.24,72.5,flight,NaN,12.588875,52.667995,True,GPS,...,NaN,NaN,0,0,NaN,NaN,3089_139537871,NaN,NaN,POINT (12.58887 52.66799)
2013-09-29 17:40:06,1149963613,2013-09-29 17:40:06,259.52,228.7,flight,NaN,12.483441,52.607326,True,GPS,...,NaN,NaN,0,0,NaN,NaN,3089_139537871,NaN,NaN,POINT (12.48344 52.60733)


In [ ]:
geopandas.hvplot()

In [ ]:
pickle.dump(geopandas, open('./geopandas.pickle', 'wb'))
geopandas_loaded = pickle.load(open('./geopandas.pickle', 'rb'))
geopandas_loaded

In [ ]:
pickle.dump(movingpandas, open('./movingpandas.pickle', 'wb'))
movingpandas_loaded = pickle.load(open('./movingpandas.pickle', 'rb'))
movingpandas_loaded

In [ ]:
movingpandas_loaded.hvplot()

In [ ]:
geopandas_loaded.hvplot()